In [28]:
%load_ext autoreload
%autoreload 2

import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

from src.loader import load_csv
import pandas as pd


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
df = load_csv("../data/raw/gapminder_full.csv")
df.head()

[INFO] loaded 1704 rows and 6 columns
[INFO] Columns: Index(['country', 'year', 'population', 'continent', 'life_exp', 'gdp_cap'], dtype='object')


,country,year,population,continent,life_exp,gdp_cap
0,Afghanistan,1952,8425333,Asia,28.801,779.445314
1,Afghanistan,1957,9240934,Asia,30.332,820.853030
2,Afghanistan,1962,10267083,Asia,31.997,853.100710
3,Afghanistan,1967,11537966,Asia,34.020,836.197138
4,Afghanistan,1972,13079460,Asia,36.088,739.981106


In [30]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1704 entries, 0 to 1703
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     1704 non-null   object 
 1   year        1704 non-null   int64  
 2   population  1704 non-null   int64  
 3   continent   1704 non-null   object 
 4   life_exp    1704 non-null   float64
 5   gdp_cap     1704 non-null   float64
dtypes: float64(2), int64(2), object(2)
memory usage: 80.0+ KB
None


In [31]:
print(df.describe())

             year    population     life_exp        gdp_cap
count  1704.00000  1.704000e+03  1704.000000    1704.000000
mean   1979.50000  2.960121e+07    59.474439    7215.327081
std      17.26533  1.061579e+08    12.917107    9857.454543
min    1952.00000  6.001100e+04    23.599000     241.165876
25%    1965.75000  2.793664e+06    48.198000    1202.060309
50%    1979.50000  7.023596e+06    60.712500    3531.846988
75%    1993.25000  1.958522e+07    70.845500    9325.462346
max    2007.00000  1.318683e+09    82.603000  113523.132900


In [32]:
print(df.isnull().sum())

country       0
year          0
population    0
continent     0
life_exp      0
gdp_cap       0
dtype: int64


In [33]:
print(df.dtypes)

country        object
year            int64
population      int64
continent      object
life_exp      float64
gdp_cap       float64
dtype: object


In [34]:
from src.cleaning import convert_column_to_datetime, standardize_text_column

df = convert_column_to_datetime(df, 'year', '%Y')
print(df.head())

[INFO] Converted column 'year' to datetime with format '%Y'.
       country       year  population continent  life_exp     gdp_cap
0  Afghanistan 1952-01-01     8425333      Asia    28.801  779.445314
1  Afghanistan 1957-01-01     9240934      Asia    30.332  820.853030
2  Afghanistan 1962-01-01    10267083      Asia    31.997  853.100710
3  Afghanistan 1967-01-01    11537966      Asia    34.020  836.197138
4  Afghanistan 1972-01-01    13079460      Asia    36.088  739.981106


In [35]:
df['gdp_total'] = df['gdp_cap'] * df['population']
df['life_exp_bin'] = pd.cut(df["life_exp"], bins= [0, 50, 65, 80, 90], labels=["Low", "Medium", "High", "Very High"])
print(df.head())

       country       year  population  ...     gdp_cap     gdp_total  life_exp_bin
0  Afghanistan 1952-01-01     8425333  ...  779.445314  6.567086e+09           Low
1  Afghanistan 1957-01-01     9240934  ...  820.853030  7.585449e+09           Low
2  Afghanistan 1962-01-01    10267083  ...  853.100710  8.758856e+09           Low
3  Afghanistan 1967-01-01    11537966  ...  836.197138  9.648014e+09           Low
4  Afghanistan 1972-01-01    13079460  ...  739.981106  9.678553e+09           Low

[5 rows x 8 columns]


In [36]:
df = standardize_text_column(df, ["country", "continent"], False)

[INFO] Standardized text in column 'country'.
[INFO] Standardized text in column 'continent'.


In [37]:
print(df.head())

       country       year  population  ...     gdp_cap     gdp_total  life_exp_bin
0  Afghanistan 1952-01-01     8425333  ...  779.445314  6.567086e+09           Low
1  Afghanistan 1957-01-01     9240934  ...  820.853030  7.585449e+09           Low
2  Afghanistan 1962-01-01    10267083  ...  853.100710  8.758856e+09           Low
3  Afghanistan 1967-01-01    11537966  ...  836.197138  9.648014e+09           Low
4  Afghanistan 1972-01-01    13079460  ...  739.981106  9.678553e+09           Low

[5 rows x 8 columns]
